## Overlays maps of p-values.

We assume the p-value maps have already been saved as tiff files.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
import tifffile

from janelia_core.visualization.volume_visualization import make_rgb_three_ch_z_plane_movie

In [3]:
%matplotlib qt

In [12]:
# Paths to .tiff files to generate maps for
r_file = r'A:\projects\keller_vnc\results\whole_brain_stats\v10_organized\before_reporting\T\T_before_reporting_ref_O_cut_off_time_5_4997_mt_A4_dff_1_5_5_long_bl_p_vls.tiff'
g_file = None #r'A:\projects\keller_vnc\results\whole_brain_stats\v10_organized\after_reporting\F\F_after_reporting_ref_O_cut_off_time_5_4997_mt_A4_dff_1_5_5_long_bl_p_vls.tiff'
b_file = r'A:\projects\keller_vnc\results\whole_brain_stats\v10_organized\after_reporting\T\T_after_reporting_ref_O_cut_off_time_5_4997_mt_A4_dff_1_5_5_long_bl_p_vls.tiff'


# Labels to associate with each color
r_label = 'Before T'
g_label = '' #'Forward'
b_label = 'After T'


# Specify max p-value that receives a non-black color
max_p_vl = .05

# Specify lower percentile we use for choosing p-values that colors saturate at (values should be between 0 and 100)
min_p_val_percentile = 1.0

# Folder to save results in 
save_folder = r'A:\projects\keller_vnc\results\whole_brain_stats\overlays'
save_name = 'before_v_after_T_1_5_5_co5_4997.mp4'

## Load the maps

In [13]:
maps = [tifffile.imread(f) if f is not None else None for f in [r_file, g_file, b_file] ]

## Convert the maps to log values

In [14]:
log_maps = [np.log10(m) if m is not None else None for m in maps]

## Merge the maps into an RGB color image

In [15]:
for m in log_maps:
    if m is not None:
        map_shape = m.shape
        
rgb_map_shape = list(map_shape) + [3]

In [16]:
labels = [r_label, g_label, b_label]

rgb_map = np.zeros(rgb_map_shape)
cmaps = [None]*3

for m_i, m in enumerate(log_maps):
    if m is not None:
        p_min = np.nanpercentile(m, min_p_val_percentile)
        p_max = np.log10(max_p_vl)
        p_range = p_max - p_min
    
        norm_m = 1 - (m - p_min)/p_range
        norm_m[norm_m > 1.0] = 1.0
        norm_m[norm_m < 0.0] = 0.0
        norm_m[np.isnan(norm_m)] = 0.0
    
        rgb_map[:,:,:, m_i] = norm_m
        
        cmaps[m_i] = {'label': labels[m_i], 'dark_vl': p_max, 'bright_vl': p_min}
    
    

## Generate movie

In [17]:
rgb_planes = [rgb_map[z_i, :,:,:] for z_i in range(rgb_map.shape[0])]

In [18]:
save_path = str(Path(save_folder) / save_name)

In [19]:
make_rgb_three_ch_z_plane_movie(z_imgs=rgb_planes, save_path=save_path, 
                                cmaps=cmaps)